In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import pandas as  pd

In [ ]:
# Access url 
website = 'https://prims.pharmacyboardkenya.org/pharma_register_r_public/?'
path = r"C:\Program Files (x86)\chromedriver-win64\chromedriver.exe"

driver = webdriver.Chrome(path)
driver.get(website)
driver.maximize_window()

# Set number of rows per page
items = Select(driver.find_element_by_xpath('//*[@id="DataTables_Table_0_length"]/label/select'))
items.select_by_visible_text('100')

time.sleep(2)

# Extract table headers
table_headers = []

headers = driver.find_elements_by_tag_name('th')

for header in headers:
    table_headers.append(header.text)
    

# Extract records

products =[]

for i in range(1, 134):
    
    print("Scraping page", i)
    
    rows = driver.find_elements_by_xpath("//tr[@class ='odd' or @class='even']")
    
    for row in rows:
        cell = row.find_elements_by_tag_name('td')
        cell_data =[data.text.lower() for data in cell]
        products.append(cell_data)
        
    # Scroll the 'next' button into view and click it
    next_button = driver.find_element_by_xpath( '//a[@data-dt-idx="8"]')
    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
    time.sleep(1)  
        
    # Click the 'next' button
    try:
        next_button.click()
    except Exception as e:
        print(f"Error clicking next button page {i}: {e}")
        break
        
    time.sleep(2)  
    
# Create and export dataframe       
df = pd.DataFrame(products, columns= table_headers)
df.to_csv(r"C:\Users\user\Desktop\Python Project\registered_products_ppb_2024.csv")

# Close webpage
driver.quit()

In [2]:
# Import csv and create dataframe
products = pd.read_csv(r"C:\Users\user\Desktop\Python Project\registered_products_ppb_2024.csv", index_col= 0)

products.head()

,Registration No,TradeName,INN of API,Dosage,PackSize,Application Type,Application Path,Local/Foreign,Registration Date,LTR,MAH,Status
0,h2019/4480/584er,zyvir 1,entecavir,1mg per tablet,blister pack of 10’s (pack size: 3x10 tablets),generic/biosimilars,national (full assesment),foreign,2019-02-21 00:00:00,cadila healthcare limited,cadila healthcare limited,registered
1,h2019/4479/583er,zyvir 0.5,entecavir,0.5mg per tablet,blister pack of 10’s (pack size: 3x10 tablets),generic/biosimilars,national (full assesment),foreign,2019-02-21 00:00:00,cadila healthcare limited,cadila healthcare limited,registered
2,7717,zytix tablet,abiraterone acetate usp,each tablet contains: 250 mg of abiraterone ac...,1x6’s in alu-alu blister with an insert.,generic/biosimilars,national (full assesment),foreign,2020-09-09 00:00:00,generics africa ltd,beacon medicare ltd,registered
3,h2017/ctd3785/116,zytiga,abiraterorone acetate,250mg,each hdpe bottle has a child-resistant closure...,new/innovator,national (full assesment),foreign,NaN,phillips therapeutics limited,janssen-cilag international nv,registered
4,h2006/181,zytee rb gel,choline salicylate benzalkonium chloride,choline salicylate 9.0 %w/v benzalkonium chlor...,NaN,NaN,national (full assesment),foreign,NaN,pan pharmaceuticals ltd,NaN,registered


In [128]:
# Explore df
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13232 entries, 0 to 13231
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Registration No    13215 non-null  object
 1   TradeName          13232 non-null  object
 2   INN of API         13188 non-null  object
 3   Dosage             13135 non-null  object
 4   PackSize           5043 non-null   object
 5   Application Type   5057 non-null   object
 6   Application Path   13232 non-null  object
 7   Local/Foreign      13232 non-null  object
 8   Registration Date  1468 non-null   object
 9   LTR                13232 non-null  object
 10  MAH                4911 non-null   object
 11  Status             13232 non-null  object
dtypes: object(12)
memory usage: 1.3+ MB


In [ ]:
# Remove columns not needed in the analysis
products.drop(['MAH', 'PackSize', 'Application Type', 'Registration Date'], axis= 'columns', inplace=True)

In [130]:
# Print revised df
products.head()

,Registration No,TradeName,INN of API,Dosage,Application Path,Local/Foreign,LTR,Status
0,h2019/4480/584er,zyvir 1,entecavir,1mg per tablet,national (full assesment),foreign,cadila healthcare limited,registered
1,h2019/4479/583er,zyvir 0.5,entecavir,0.5mg per tablet,national (full assesment),foreign,cadila healthcare limited,registered
2,7717,zytix tablet,abiraterone acetate usp,each tablet contains: 250 mg of abiraterone ac...,national (full assesment),foreign,generics africa ltd,registered
3,h2017/ctd3785/116,zytiga,abiraterorone acetate,250mg,national (full assesment),foreign,phillips therapeutics limited,registered
4,h2006/181,zytee rb gel,choline salicylate benzalkonium chloride,choline salicylate 9.0 %w/v benzalkonium chlor...,national (full assesment),foreign,pan pharmaceuticals ltd,registered


In [132]:
# Rename columns

products.rename(columns = {'Registration No': 'Registration_no','TradeName': 'Trade_name','Local/Foreign':'Source',
                          'INN of API': 'INN','Application Path':'Application_path',}, inplace=True)

In [134]:
products.head(5)

,Registration_no,Trade_name,INN,Dosage,Application_path,Source,LTR,Status
0,h2019/4480/584er,zyvir 1,entecavir,1mg per tablet,national (full assesment),foreign,cadila healthcare limited,registered
1,h2019/4479/583er,zyvir 0.5,entecavir,0.5mg per tablet,national (full assesment),foreign,cadila healthcare limited,registered
2,7717,zytix tablet,abiraterone acetate usp,each tablet contains: 250 mg of abiraterone ac...,national (full assesment),foreign,generics africa ltd,registered
3,h2017/ctd3785/116,zytiga,abiraterorone acetate,250mg,national (full assesment),foreign,phillips therapeutics limited,registered
4,h2006/181,zytee rb gel,choline salicylate benzalkonium chloride,choline salicylate 9.0 %w/v benzalkonium chlor...,national (full assesment),foreign,pan pharmaceuticals ltd,registered


In [135]:
# Check nulls for local products
local_products = products[products['Source'] =='local']
local_products[local_products.isna().any(axis=1)]

# Why are all the products with null registration_no from Elys?

,Registration_no,Trade_name,INN,Dosage,Application_path,Source,LTR,Status
5755,NaN,lyfe ors,glucose bp sodium chloride bp sodium citrate d...,13.5 g 2.6 g 2.9 g 1.5 g / liter,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
10567,NaN,ciflo-250 tablets,ciprofloxacin hydrochloride bp,250 mg / tablet,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
10568,NaN,ciflo tablets 500 mg,ciprofloxacin hydrochloride bp,500 mg,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
10584,NaN,chlorpromazine-100 tablets,chlorpromazine,100 mg,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
10598,NaN,chlorpheniramine maleate syrup,chlorpheniramine maleate bp,2 mg / 5ml,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
10622,NaN,chestcof tablets,liquorice liquid extract (glycyrrhiza) bp ment...,liquorice liquid extract (glycyrrhiza) bp 35 m...,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
10623,NaN,chestcof syrup,"liquorice liquid extract bp, menthol bp, anise...","liquorice liquid extract bp 1 ml, menthol bp 4...",national (full assesment),local,elys chemical industries ltd - enterprise road,registered
11017,NaN,cardi-80 tablets,acetylsalicylic acid bp,80 mg,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
11077,NaN,candistat-200 vaginal tablets,clotrimazole,200 mg/ tablet,national (full assesment),local,elys chemical industries ltd - enterprise road,registered
12157,NaN,aspirin tablets,acetylsalicylic acid bp,300 mg,national (full assesment),local,elys chemical industries ltd - enterprise road,registered


In [178]:
# Extract APIs/INNs for locally manufactured products
unique_local = local_products['INN'].unique()

# Create df of unique local INNs/APIs
unique_local_products = pd.DataFrame(unique_local, columns=['INN'])

# Export df to Excel for hand cleaning
unique_local_products.to_excel(r"C:\Users\user\Desktop\Python Project\unique_local_products.xlsx", index=False)



In [3]:
# Create df of cleaned local products
clean_local_products = pd.read_excel(r"C:\Users\user\Desktop\Python Project\unique_local_products.xlsx")
clean_local_products.head()

,INN
0,rosuvastatin
1,ibuprofen
2,cefuroxime
3,azithromycin
4,zinc sulfate


In [4]:
# Load Kenya essential medicines list (extracted with Power Query)
wb = pd.ExcelFile(r"C:\Users\user\Desktop\Python Project\keml_corrected.xlsx")
essential_drugs = pd.read_excel(wb, 'Append1')
essential_drugs

,Name of Medicine,Dose-form,Strength / Size
0,Halothane,Inhalation,250mL
1,Isoflurane,Inhalation,250mL
2,Sevoflurane^{2},Inhalation,250mL
3,Dexmedetomidine^{3},Injection,200 micrograms/2mL
4,Etomidate^{4},Injection,2mg/mL (10mL Vial)
...,...,...,...
1457,NaN,NaN,25 mCi/mL
1458,Lutetium-177 oxodotreotide\n(dotatate),Injection,370 MBq/mL (10 mCi/mL)
1459,NaN,NaN,35� MEDICINES for BENIGN PROSTATIC HYPERPLASIA...
1460,Finasteride,Tablet,5mg


In [5]:
# View null values
essential_drugs[essential_drugs.isna().any(axis=1)]

,Name of Medicine,Dose-form,Strength / Size
14,NaN,Topical spray,2% (as HCl)
15,NaN,NaN,4% (as HCl)
16,NaN,NaN,10% (as HCl)
18,NaN,Injection^{12},"2% (HCl or sulphate) +\n1:200,000 in vial"
26,NaN,NaN,5mg (as HCl)/mL (3mL\namp)^{21}
...,...,...,...
1450,Technetium-99m sodium\npertechnetate,"Precursor\nradiopharmaceutical, eluted\nfrom a...",NaN
1451,[Fluorine-18] Prostate\nSpecific Membrane Anti...,NaN,NaN
1455,NaN,NaN,34.2 Therapeutic Radiopharmaceuticals
1457,NaN,NaN,25 mCi/mL


In [6]:
# Null values due to ditto marks; fill with ffill
essential_drugs.fillna(method='ffill', inplace=True)
essential_drugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name of Medicine  1462 non-null   object
 1   Dose-form         1462 non-null   object
 2   Strength / Size   1462 non-null   object
dtypes: object(3)
memory usage: 34.4+ KB


In [7]:
# Remove section headers in dosage column
essential_drugs_clean = essential_drugs[~essential_drugs['Dose-form'].str.contains('^\d')]


In [8]:
# Create essential drugs list by splitting name of medicine from carat 
essential_drugs_list = list(essential_drugs_clean['Name of Medicine'].apply(lambda x: x.split('^')[0]))

# Remove duplicates
essential_drugs_set = set(essential_drugs_list)

# Create essential_drugs_df
kenya_essential_drugs = pd.DataFrame(list(essential_drugs_set), columns=['INN'])
kenya_essential_drugs

,INN
0,Calamine
1,Etanercept
2,Malaria vaccine
3,Tetracaine
4,Zinc Hyaluronate (zinc-hyaluronan)
...,...
701,Clofazamine
702,Enoxaparin
703,Misoprostol
704,Octreotide


In [12]:
# Export list of essential drugs for hand cleaning
kenya_essential_drugs.to_excel(r"C:\Users\user\Desktop\Python Project\kesdl_extract.xlsx", index=False)

In [4]:
# Load cleaned kenya essential drugs
clean_essential_drugs = pd.read_excel(r"C:\Users\user\Desktop\Python Project\kesdl_extract.xlsx")
clean_essential_drugs.head()

,INN
0,Filgrastim
1,Tenofovir disoproxil fumarate/Emtricitabine
2,Lignocaine
3,Glucose
4,Rifampicin/Isoniazid/Pyrazinamide


In [5]:
# Convert INN to lowercase
clean_essential_drugs['INN'] = clean_essential_drugs['INN'].apply(lambda x: x.lower())

In [6]:
# Extract essential drugs manufactured locally
essential_locally_manufactured = clean_essential_drugs[clean_essential_drugs['INN'].isin(list(clean_local_products['INN']))]

# Export list of essential drugs manufactured locally
essential_locally_manufactured.to_excel(r"C:\Users\user\Desktop\Python Project\locally_manufactured_keml.xlsx",
                                       index=False)

In [7]:
# Extract essential drugs not manufactured locally
essential_imported = clean_essential_drugs[~clean_essential_drugs['INN'].isin(list(clean_local_products['INN']))]

# Export list of essential drugs not manufactured locally
essential_imported.to_excel(r"C:\Users\user\Desktop\Python Project\essential_imported_keml.xlsx",
                                       index=False)